## 2,4호선 데이터 정제

#### 배차시간 데이터 확인

In [1]:
#### 배차시간 데이터 확인
import pandas as pd
subway_time = pd.read_csv("../Data/지하철배차시간데이터/서울교통공사_서울 도시철도 열차운행시각표_20240305.csv", encoding='euc-kr')
#서울교통공사_서울 도시철도 열차운행시각표_20240305.csv

/var/folders/5b/bk7dwjqs57d1fzmtyxy952540000gn/T/ipykernel_90239/1770200333.py:3: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  subway_time = pd.read_csv("../Data/지하철배차시간데이터/서울교통공사_서울 도시철도 열차운행시각표_20240305.csv", encoding='euc-kr')


In [2]:
subway_time[subway_time['호선'] == 3]
subway_time[subway_time['역사코드'] == 342.0]

,고유번호,호선,역사코드,역사명,주중주말,방향,급행여부,열차코드,열차도착시간,열차출발시간,출발역,도착역
27760,27761,3,342.0,오금,DAY,DOWN,0,5981K,05:20:30,NaN,수서,오금
27761,27762,3,342.0,오금,DAY,DOWN,0,3001,06:02:30,NaN,압구정,오금
27762,27763,3,342.0,오금,DAY,DOWN,0,3003,06:21:00,NaN,독립문,오금
27763,27764,3,342.0,오금,DAY,DOWN,0,3005,06:34:00,NaN,구파발,오금
27764,27765,3,342.0,오금,DAY,DOWN,0,3007,06:42:00,NaN,구파발,오금
...,...,...,...,...,...,...,...,...,...,...,...,...
286263,286264,3,342.0,오금,SAT,DOWN,0,S3335,23:10:00,NaN,구파발,오금
286264,286265,3,342.0,오금,SAT,DOWN,0,K3343,23:40:00,NaN,대화,오금
286265,286266,3,342.0,오금,SAT,DOWN,0,S3339,23:25:00,NaN,구파발,오금
286266,286267,3,342.0,오금,SAT,DOWN,0,S3349,23:53:30,NaN,대화,오금


### 열차 출발시간, 도착시간에 NaN 값 존재!
기준을 출발로 잡아야 하나, 도착으로 잡아야 하나?


In [3]:
subway_time.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 373624 entries, 0 to 373623
Data columns (total 12 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   고유번호    373624 non-null  int64  
 1   호선      373624 non-null  int64  
 2   역사코드    373131 non-null  float64
 3   역사명     373624 non-null  object 
 4   주중주말    373624 non-null  object 
 5   방향      373624 non-null  object 
 6   급행여부    373624 non-null  int64  
 7   열차코드    373624 non-null  object 
 8   열차도착시간  357878 non-null  object 
 9   열차출발시간  349446 non-null  object 
 10  출발역     373624 non-null  object 
 11  도착역     373624 non-null  object 
dtypes: float64(1), int64(3), object(8)
memory usage: 34.2+ MB


#### 열차 도착시간의 nonnulll값이 출발보다 크므로 도착시간 기준으로 배차간격 계산!

In [4]:
#  열차 도착시간의 NaN 값을 열차출발시간 값으로 대체
subway_time['열차도착시간'] = subway_time['열차도착시간'].fillna(subway_time['열차출발시간'])

In [5]:

subway_time.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 373624 entries, 0 to 373623
Data columns (total 12 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   고유번호    373624 non-null  int64  
 1   호선      373624 non-null  int64  
 2   역사코드    373131 non-null  float64
 3   역사명     373624 non-null  object 
 4   주중주말    373624 non-null  object 
 5   방향      373624 non-null  object 
 6   급행여부    373624 non-null  int64  
 7   열차코드    373624 non-null  object 
 8   열차도착시간  373624 non-null  object 
 9   열차출발시간  349446 non-null  object 
 10  출발역     373624 non-null  object 
 11  도착역     373624 non-null  object 
dtypes: float64(1), int64(3), object(8)
memory usage: 34.2+ MB


## 비상비상비상!!!!! 역사코드에도 Nan값이 있네?


In [6]:
# '역사코드' 열에서 NaN 값만 출력
subway_time[subway_time['역사코드'].isna()]['역사명'].unique()

array(['마전', '남위례'], dtype=object)

# 역사 코드 없는 역은 남위례(8호선), 마전(인천 2호선)
-> 교통공사 데이터 확인후 남위례의 역 번호가 2828 인것을 확인, 마전은 필요없는 데이터
### 남위례의 역 번호를 2828로 대체필요!

In [26]:
# subway_time[subway_time['역사명'] == '남위례']['역사코드'] = 2828

In [8]:
subway_time[subway_time['역사명'] == '남위례']

,고유번호,호선,역사코드,역사명,주중주말,방향,급행여부,열차코드,열차도착시간,열차출발시간,출발역,도착역
329174,329175,8,NaN,남위례,SAT,UP,0,8122,14:00:30,14:01:00,모란,암사
329175,329176,8,NaN,남위례,SAT,UP,0,8124,14:09:00,14:09:30,모란,암사
329176,329177,8,NaN,남위례,SAT,UP,0,8126,14:17:30,14:18:00,모란,암사
329177,329178,8,NaN,남위례,SAT,UP,0,8128,14:26:00,14:26:30,모란,암사
329178,329179,8,NaN,남위례,SAT,UP,0,8130,14:34:30,14:35:00,모란,암사
...,...,...,...,...,...,...,...,...,...,...,...,...
331333,331334,8,NaN,남위례,SAT,DOWN,0,8241,23:10:00,23:10:30,암사,모란
331334,331335,8,NaN,남위례,SAT,DOWN,0,8243,23:21:00,23:21:30,암사,모란
331335,331336,8,NaN,남위례,SAT,DOWN,0,8245,23:32:00,23:32:30,암사,모란
331336,331337,8,NaN,남위례,SAT,DOWN,0,8247,23:44:00,23:44:30,암사,모란


In [9]:
import pandas as pd ,numpy as  np # Data manipulatioin
import seaborn as sns,matplotlib.pyplot as plt # visiulization
import warnings;warnings.filterwarnings('ignore') # 경고 무시 
# 기본 세팅
def plotSetting(pltStyle):
    import matplotlib.pyplot as plt # visiulization
    import platform
    from matplotlib import font_manager, rc # rc : 폰트 변경 모듈font_manager : 폰트 관리 모듈
    plt.style.use(pltStyle)
    plt.rcParams['axes.unicode_minus'] = False# unicode 설정
    if platform.system() == 'Darwin': rc('font', family='AppleGothic') # os가 macos
    elif platform.system() == 'Windows': # os가 windows
        path = 'c:/Windows/Fonts/malgun.ttf' 
        font_name = font_manager.FontProperties(fname=path).get_name()
        rc('font', family=font_name)
    else:
        print("Unknown System")
    print("___## OS platform 한글 세팅완료 ## ___")
# graph style seaborn
plotSetting("seaborn-v0_8")


___## OS platform 한글 세팅완료 ## ___


In [10]:
# 서울교통공사_역별 일별 시간대별 승하차인원 정보_23.11_24.01.csv

# Data loading 
subway = pd.read_csv("../Data/교통공사데이터/서울교통공사_역별 일별 시간대별 승하차인원 정보_23.11_24.01.csv", encoding='euc-kr')
subway.head()

,연번,날짜,호선,역번호,역명,구분,06시 이전,06시-07시,07시-08시,08시-09시,...,15시-16시,16시-17시,17시-18시,18시-19시,19시-20시,20시-21시,21시-22시,22시-23시,23시-24시,24시 이후
0,1,2023-11-01,1호선,150,서울역,승차,342,520,1929,2864,...,3265,3834,6739,9519,4370,2870,2550,1641,753,72
1,2,2023-11-01,1호선,150,서울역,하차,300,2166,5186,10500,...,2372,2231,3041,3053,1729,1249,1114,734,381,108
2,3,2023-11-01,1호선,151,시청,승차,86,157,303,415,...,1973,2508,4322,7531,2677,2211,2100,1250,399,31
3,4,2023-11-01,1호선,151,시청,하차,220,1014,3464,9344,...,971,973,1102,1012,556,321,264,174,108,40
4,5,2023-11-01,1호선,152,종각,승차,144,167,256,454,...,2364,2805,6649,10098,3847,3149,3366,2330,908,38


In [11]:
subway[subway['역명'] == '남위례']

,연번,날짜,호선,역번호,역명,구분,06시 이전,06시-07시,07시-08시,08시-09시,...,15시-16시,16시-17시,17시-18시,18시-19시,19시-20시,20시-21시,21시-22시,22시-23시,23시-24시,24시 이후
544,545,2023-11-01,8호선,2828,남위례,승차,66,289,1035,1552,...,311,513,667,812,288,229,242,197,70,35
545,546,2023-11-01,8호선,2828,남위례,하차,38,128,337,1259,...,329,385,628,1125,800,411,367,340,193,71
1090,1091,2023-11-02,8호선,2828,남위례,승차,62,289,988,1254,...,337,531,679,839,325,246,262,226,78,26
1091,1092,2023-11-02,8호선,2828,남위례,하차,37,136,338,1221,...,406,442,661,1113,781,419,392,383,211,76
1636,1637,2023-11-03,8호선,2828,남위례,승차,62,263,939,1137,...,370,591,707,815,351,265,255,185,103,47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49143,49144,2024-01-29,8호선,2828,남위례,하차,41,127,313,921,...,314,356,557,1136,783,394,371,328,155,63
49688,49689,2024-01-30,8호선,2828,남위례,승차,62,300,873,1104,...,330,394,605,809,317,288,233,165,74,38
49689,49690,2024-01-30,8호선,2828,남위례,하차,37,131,306,954,...,350,439,588,1022,794,421,421,317,202,96
50236,50237,2024-01-31,8호선,2828,남위례,승차,54,283,923,1111,...,379,392,613,789,338,263,251,202,105,33


In [12]:
# 2,4 호선 정보 빼기
line_3=subway[subway['호선'] == '3호선']
line_4=subway[subway['호선'] == '4호선']

In [13]:
line_3.tail()

#2023-11-01 ~ 2024-01-31 사이의 데이터

,연번,날짜,호선,역번호,역명,구분,06시 이전,06시-07시,07시-08시,08시-09시,...,15시-16시,16시-17시,17시-18시,18시-19시,19시-20시,20시-21시,21시-22시,22시-23시,23시-24시,24시 이후
49873,49874,2024-01-31,3호선,340,가락시장,하차,22,257,555,1200,...,577,528,802,1079,731,418,409,394,172,73
49874,49875,2024-01-31,3호선,341,경찰병원,승차,94,302,972,1176,...,412,434,608,926,312,241,200,141,36,7
49875,49876,2024-01-31,3호선,341,경찰병원,하차,9,124,408,1347,...,264,389,620,950,696,400,378,337,203,57
49876,49877,2024-01-31,3호선,342,오금,승차,155,357,1211,1401,...,221,326,435,540,173,143,99,91,79,4
49877,49878,2024-01-31,3호선,342,오금,하차,1,113,199,585,...,303,324,617,936,726,457,349,467,218,77


In [14]:
#2호선 데이터 확인
def dataInfoProcessing(line_3, replace_Nan=False):
        
    print("\n1. Data colum numbers : ",len(line_3.columns))
    #print(subway.dolumns3)
    #print(subway.info())
    null_message =f"총 {line_3.isnull().sum().sum()}개의 null 이 있습니다!" if line_3.isnull().sum().sum() else "Null 없는 clean data!"
    print("\n2. null ceck 결과",null_message)
    ### Null 이 있는 칼럼 추출
    haveNullColumn =[]
    for idx, col in enumerate(line_3.columns):
        if line_3[f"{col}"].isnull().sum():
            print(f"   => {idx}번째.[{col}]컬럼 : ",f"null {line_3[f'{col}'].isnull().sum()} 개,\t not null {line_3[f'{col}'].notnull().sum()} 개")
            ## Null data fill
    if replace_Nan : ## nan 을 0 으로 대체 
        line_3[col].fillna(value=0, inplace=True)  
    
    print("\n3. Column  Information (중복체크)")
    print( "\tidx.columName |\t\t\t\t |Colum Info(dtype)|** ")
    print( "\t","--"*len("columIdx |\t\t\t\t **|Col(dtype)|** "))
    for idx, col in enumerate(line_3.dtypes.keys()):
        if idx< 7:
            if len(f"\t{idx}.[{col}({line_3.dtypes[col]})]:")<20:
                print(f"\t{idx}.[{col}({line_3.dtypes[col]})]:",\
                    f"{len(line_3[col].unique())}/{len(line_3[col])} [uniq/raw]", sep=" \t\t\t")
            else:
                    print(f"\t{idx}.[{col}({line_3.dtypes[col]})]:",\
                    f"{len(line_3[col].unique())}/{len(line_3[col])} [uniq/raw]", sep=" \t\t")

    else: 
        print(f"\t ...etc (추가로 {len(line_3.dtypes.keys())-5}개의 칼럼이 있습니다 )")
   
              
dataInfoProcessing(subway)

# subway.info(show_counts=False)
# subway.value_counts('수송일자')






1. Data colum numbers :  26

2. null ceck 결과 Null 없는 clean data!

3. Column  Information (중복체크)
	idx.columName |				 |Colum Info(dtype)|** 
	 ----------------------------------------------------------------
	0.[연번(int64)]: 			50238/50238 [uniq/raw]
	1.[날짜(object)]: 			92/50238 [uniq/raw]
	2.[호선(object)]: 			8/50238 [uniq/raw]
	3.[역번호(int64)]: 			278/50238 [uniq/raw]
	4.[역명(object)]: 			244/50238 [uniq/raw]
	5.[구분(object)]: 			2/50238 [uniq/raw]
	6.[06시 이전(int64)]: 			1248/50238 [uniq/raw]
	 ...etc (추가로 21개의 칼럼이 있습니다 )


In [15]:
#2호선 데이터 확인
def dataInfoProcessing(line_3, replace_Nan=False):
        
    print("\n1. Data colum numbers : ",len(line_3.columns))
    #print(subway.columns)
    #print(subway.info())
    null_message =f"총 {line_4.isnull().sum().sum()}개의 null 이 있습니다!" if line_4.isnull().sum().sum() else "Null 없는 clean data!"
    print("\n2. null ceck 결과",null_message)
    ### Null 이 있는 칼럼 추출
    haveNullColumn =[]
    for idx, col in enumerate(line_4.columns):
        if line_4[f"{col}"].isnull().sum():
            print(f"   => {idx}번째.[{col}]컬럼 : ",f"null {line_4[f'{col}'].isnull().sum()} 개,\t not null {line_4[f'{col}'].notnull().sum()} 개")
            ## Null data fill
    if replace_Nan : ## nan 을 0 으로 대체 
        line_4[col].fillna(value=0, inplace=True)  
    
    print("\n3. Column  Information (중복체크)")
    print( "\tidx.columName |\t\t\t\t |Colum Info(dtype)|** ")
    print( "\t","--"*len("columIdx |\t\t\t\t **|Col(dtype)|** "))
    for idx, col in enumerate(line_4.dtypes.keys()):
        if idx< 7:
            if len(f"\t{idx}.[{col}({line_4.dtypes[col]})]:")<20:
                print(f"\t{idx}.[{col}({line_4.dtypes[col]})]:",\
                    f"{len(line_4[col].unique())}/{len(line_4[col])} [uniq/raw]", sep=" \t\t\t")
            else:
                    print(f"\t{idx}.[{col}({line_4.dtypes[col]})]:",\
                    f"{len(line_4[col].unique())}/{len(line_4[col])} [uniq/raw]", sep=" \t\t")

    else: 
        print(f"\t ...etc (추가로 {len(line_4.dtypes.keys())-5}개의 칼럼이 있습니다 )")
   
              
dataInfoProcessing(subway)

# subway.info(show_counts=False)
# subway.value_counts('수송일자')






1. Data colum numbers :  26

2. null ceck 결과 Null 없는 clean data!

3. Column  Information (중복체크)
	idx.columName |				 |Colum Info(dtype)|** 
	 ----------------------------------------------------------------
	0.[연번(int64)]: 			4784/4784 [uniq/raw]
	1.[날짜(object)]: 			92/4784 [uniq/raw]
	2.[호선(object)]: 			1/4784 [uniq/raw]
	3.[역번호(int64)]: 			26/4784 [uniq/raw]
	4.[역명(object)]: 			26/4784 [uniq/raw]
	5.[구분(object)]: 			2/4784 [uniq/raw]
	6.[06시 이전(int64)]: 			738/4784 [uniq/raw]
	 ...etc (추가로 21개의 칼럼이 있습니다 )


### 위의 결과로 보아2023-11-01 ~ 2024-01-31 사이의 데이터 셋에는 null 값이나 특이점이 있는 데이터가 없음!

In [16]:

line_3.head()

,연번,날짜,호선,역번호,역명,구분,06시 이전,06시-07시,07시-08시,08시-09시,...,15시-16시,16시-17시,17시-18시,18시-19시,19시-20시,20시-21시,21시-22시,22시-23시,23시-24시,24시 이후
120,121,2023-11-01,3호선,309,지축,승차,145,474,1485,1321,...,381,413,416,401,232,178,136,154,51,10
121,122,2023-11-01,3호선,309,지축,하차,19,253,218,465,...,368,509,677,1247,1004,580,527,447,227,55
122,123,2023-11-01,3호선,310,구파발,승차,481,1578,4184,3983,...,1057,1326,1307,1083,683,507,492,385,132,30
123,124,2023-11-01,3호선,310,구파발,하차,57,410,983,1281,...,1066,1274,1674,3526,2764,1382,1177,1125,644,334
124,125,2023-11-01,3호선,311,연신내,승차,1029,2037,5567,6332,...,1742,1994,1664,1575,1041,821,758,623,316,67


In [17]:
line_3['역명']
# line_['역명']= "구파발"

120        지축
121        지축
122       구파발
123       구파발
124       연신내
         ... 
49873    가락시장
49874    경찰병원
49875    경찰병원
49876      오금
49877      오금
Name: 역명, Length: 6210, dtype: object

In [18]:
#새로운 역명 이름 생성
line_3['newstation'] = line_3['역명'] + "_"+line_3['역번호'].astype(str)
line_4['newstation'] = line_4['역명'] +"_"+ line_4['역번호'].astype(str)
line_4.head()

,연번,날짜,호선,역번호,역명,구분,06시 이전,06시-07시,07시-08시,08시-09시,...,16시-17시,17시-18시,18시-19시,19시-20시,20시-21시,21시-22시,22시-23시,23시-24시,24시 이후,newstation
188,189,2023-11-01,4호선,409,당고개,승차,378,724,2084,1707,...,457,437,431,245,171,135,93,58,11,당고개_409
189,190,2023-11-01,4호선,409,당고개,하차,2,204,272,393,...,619,787,1146,1055,625,540,502,300,162,당고개_409
190,191,2023-11-01,4호선,410,상계,승차,603,1682,4221,3390,...,924,922,823,509,474,385,465,146,30,상계_410
191,192,2023-11-01,4호선,410,상계,하차,17,261,764,899,...,1457,1935,2952,2669,1472,1168,1116,642,270,상계_410
192,193,2023-11-01,4호선,411,노원,승차,509,1032,2391,2536,...,1122,1196,1174,803,887,827,779,360,91,노원_411


In [19]:
#필요 없어진 역명,역이름 컬럼삭제
line_3 = line_3.drop(['역명', '역번호'], axis=1)
line_4 = line_4.drop(['역명', '역번호'], axis=1)


In [20]:
line_3.head()
line_3.columns

Index(['연번', '날짜', '호선', '구분', '06시 이전', '06시-07시', '07시-08시', '08시-09시',
       '09시-10시', '10시-11시', '11시-12시', '12시-13시', '13시-14시', '14시-15시',
       '15시-16시', '16시-17시', '17시-18시', '18시-19시', '19시-20시', '20시-21시',
       '21시-22시', '22시-23시', '23시-24시', '24시 이후', 'newstation'],
      dtype='object')

#### column 순서 재배치

In [21]:
# 현재 열 이름 리스트 가져오기
columns = line_3.columns.tolist()

# 20번째 열 이름
col_to_move = columns[24]  

# 20번째 열을 제외한 나머지 열 이름
remaining_cols = [c for c in columns if c != col_to_move]

# 새로운 열 순서 지정
new_column_order = remaining_cols[:2] + [col_to_move] + remaining_cols[2:]

# 열 순서 변경
line_3 = line_3.reindex(columns=new_column_order)
line_3.head()

,연번,날짜,newstation,호선,구분,06시 이전,06시-07시,07시-08시,08시-09시,09시-10시,...,15시-16시,16시-17시,17시-18시,18시-19시,19시-20시,20시-21시,21시-22시,22시-23시,23시-24시,24시 이후
120,121,2023-11-01,지축_309,3호선,승차,145,474,1485,1321,630,...,381,413,416,401,232,178,136,154,51,10
121,122,2023-11-01,지축_309,3호선,하차,19,253,218,465,226,...,368,509,677,1247,1004,580,527,447,227,55
122,123,2023-11-01,구파발_310,3호선,승차,481,1578,4184,3983,1968,...,1057,1326,1307,1083,683,507,492,385,132,30
123,124,2023-11-01,구파발_310,3호선,하차,57,410,983,1281,946,...,1066,1274,1674,3526,2764,1382,1177,1125,644,334
124,125,2023-11-01,연신내_311,3호선,승차,1029,2037,5567,6332,3335,...,1742,1994,1664,1575,1041,821,758,623,316,67


In [22]:
# 현재 열 이름 리스트 가져오기
columns = line_4.columns.tolist()

# 20번째 열 이름
col_to_move = columns[24]  

# 20번째 열을 제외한 나머지 열 이름
remaining_cols = [c for c in columns if c != col_to_move]

# 새로운 열 순서 지정
new_column_order = remaining_cols[:2] + [col_to_move] + remaining_cols[2:]

# 열 순서 변경
line_4 = line_3.reindex(columns=new_column_order)
line_4.head()

,연번,날짜,newstation,호선,구분,06시 이전,06시-07시,07시-08시,08시-09시,09시-10시,...,15시-16시,16시-17시,17시-18시,18시-19시,19시-20시,20시-21시,21시-22시,22시-23시,23시-24시,24시 이후
120,121,2023-11-01,지축_309,3호선,승차,145,474,1485,1321,630,...,381,413,416,401,232,178,136,154,51,10
121,122,2023-11-01,지축_309,3호선,하차,19,253,218,465,226,...,368,509,677,1247,1004,580,527,447,227,55
122,123,2023-11-01,구파발_310,3호선,승차,481,1578,4184,3983,1968,...,1057,1326,1307,1083,683,507,492,385,132,30
123,124,2023-11-01,구파발_310,3호선,하차,57,410,983,1281,946,...,1066,1274,1674,3526,2764,1382,1177,1125,644,334
124,125,2023-11-01,연신내_311,3호선,승차,1029,2037,5567,6332,3335,...,1742,1994,1664,1575,1041,821,758,623,316,67


### 3,4 호선 승하차 구분

In [23]:
#4호선 승차
line_4_on=line_4.iloc[::2, :]
#4호선 하차
line_4_off=line_4.iloc[1::2, :]
#3호선 승차
line_3_on=line_3.iloc[::2, :]
#3호선 하차
line_3_off=line_3.iloc[1::2, :]




In [24]:
line_3_on.head()

,연번,날짜,newstation,호선,구분,06시 이전,06시-07시,07시-08시,08시-09시,09시-10시,...,15시-16시,16시-17시,17시-18시,18시-19시,19시-20시,20시-21시,21시-22시,22시-23시,23시-24시,24시 이후
120,121,2023-11-01,지축_309,3호선,승차,145,474,1485,1321,630,...,381,413,416,401,232,178,136,154,51,10
122,123,2023-11-01,구파발_310,3호선,승차,481,1578,4184,3983,1968,...,1057,1326,1307,1083,683,507,492,385,132,30
124,125,2023-11-01,연신내_311,3호선,승차,1029,2037,5567,6332,3335,...,1742,1994,1664,1575,1041,821,758,623,316,67
126,127,2023-11-01,불광_312,3호선,승차,396,850,2408,2988,1509,...,888,1026,1034,1087,553,409,386,274,125,30
128,129,2023-11-01,녹번_313,3호선,승차,382,1119,3495,3638,1809,...,606,551,620,750,325,283,207,159,62,14


### 날짜 -> 월,주, 요일구분(휴일인지 아닌지)

In [25]:
line_3_on.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3105 entries, 120 to 49876
Data columns (total 25 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   연번          3105 non-null   int64 
 1   날짜          3105 non-null   object
 2   newstation  3105 non-null   object
 3   호선          3105 non-null   object
 4   구분          3105 non-null   object
 5   06시 이전      3105 non-null   int64 
 6   06시-07시     3105 non-null   int64 
 7   07시-08시     3105 non-null   int64 
 8   08시-09시     3105 non-null   int64 
 9   09시-10시     3105 non-null   int64 
 10  10시-11시     3105 non-null   int64 
 11  11시-12시     3105 non-null   int64 
 12  12시-13시     3105 non-null   int64 
 13  13시-14시     3105 non-null   int64 
 14  14시-15시     3105 non-null   int64 
 15  15시-16시     3105 non-null   int64 
 16  16시-17시     3105 non-null   int64 
 17  17시-18시     3105 non-null   int64 
 18  18시-19시     3105 non-null   int64 
 19  19시-20시     3105 non-null   int64 
 20  20시-21시   